## Summary and Explanations. ##

Title: Improving Music Genre Classification using Text Lyrics. 

Hypothesis: Lyrics does contain important information not shown in the audio itself that can improve audio classifiers.

Dataset: Custom built data set combining 
- source 1
- source 2

## Imports and load custom data-set. ##

In [11]:
import pandas as pd
import numpy as np

from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

data = pd.read_csv('DATASETS/Project_dataset.csv')

## Minor changes and some anysis ##

In [12]:
len(data)

12440

In [13]:
data.genre.value_counts()

genre
Country             1738
Ska                 1687
Blues               1469
Folk                1144
Dance                995
Rock                 917
Electronic           911
Reggae               864
Soul                 804
R&B                  469
Jazz                 431
Pop                  387
Hip-Hop              234
Indie                152
Children’s Music     122
Rap                  116
Name: count, dtype: int64

In [14]:
print('Dummy accuracy:', round(100 * 1738 / len(data)), '%')

Dummy accuracy: 14 %


In [70]:
# Encode the categorical data
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in data.columns:
    if col in ['genre', 'key', 'mode', 'time_signature']:

        data[col] = le.fit_transform(data[col])
        data[col] = data[col].astype("category")

data.head()

# for col in audio_data.columns:
#     if col in categorical_columns:
#         audio_data[col] = audio_data[col].astype("category")

,genre,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,artist_and_song,Lyrics
0,2,0.0293,0.387,234307,0.874,0.000017,3,0.1710,-4.528,0,0.0530,168.105,2,0.674,gary allan /// get off on the pain,I don't know why I love women\r\nThat love to ...
1,2,0.1730,0.633,183600,0.444,0.000003,5,0.0821,-14.090,0,0.0264,106.111,1,0.450,"hank williams, jr. /// old habits","I kicked the habit, of smoking back some time ..."
2,2,0.0600,0.493,262720,0.780,0.000248,3,0.1920,-4.127,0,0.0485,119.996,2,0.190,carrie underwood /// chaser,I need something strong tonight\nI'm needing m...
3,2,0.7730,0.687,320040,0.543,0.000002,10,0.7050,-10.151,0,0.0521,129.296,3,0.807,bobby bare /// the winner,The hulk of a man with a beer in his hand he l...
4,2,0.0655,0.618,226187,0.790,0.006410,9,0.1100,-4.973,0,0.0445,119.987,2,0.545,brooks & dunn /// proud of the house we built,I dropped to my knees in that field on your da...


In [52]:
data.genre.value_counts()

2     1738
14    1687
0     1469
5     1144
3      995
13     917
4      911
12     864
15     804
10     469
8      431
9      387
6      234
7      152
1      122
11     116
Name: genre, dtype: int64

### Audio Baseline

In [71]:
# Load the data
labels = data.genre

#print(labels)
data_audio = data.drop(columns=['genre', 'artist_and_song', 'Lyrics'])

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data_audio, labels, test_size=0.2,random_state=42)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [54]:
# Initialize the model
clf = MLPClassifier()#hidden_layer_sizes=(256,128,64), max_iter=200)

# Train the model
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# Calculate the classification report for this fold
report = classification_report(y_test, y_pred)
print(report)


              precision    recall  f1-score   support

           0       0.29      0.29      0.29       305
           1       0.05      0.05      0.05        22
           2       0.44      0.48      0.46       345
           3       0.22      0.22      0.22       205
           4       0.45      0.41      0.43       186
           5       0.19      0.21      0.20       223
           6       0.17      0.20      0.18        41
           7       0.00      0.00      0.00        26
           8       0.33      0.22      0.26        90
           9       0.11      0.16      0.13        69
          10       0.12      0.08      0.10       107
          11       0.00      0.00      0.00        22
          12       0.37      0.32      0.35       176
          13       0.16      0.15      0.15       165
          14       0.65      0.73      0.69       342
          15       0.21      0.19      0.20       164

    accuracy                           0.34      2488
   macro avg       0.23   

/home/hanbiru/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [55]:
naive_clf = GaussianNB()
naive_clf.fit(X_train, y_train)

naive_y_pred = naive_clf.predict(X_test)

# Calculate the classification report for this fold
naive_report = classification_report(y_test, naive_y_pred)
print(naive_report)

              precision    recall  f1-score   support

           0       0.42      0.05      0.08       305
           1       0.00      0.00      0.00        22
           2       0.31      0.76      0.44       345
           3       0.24      0.34      0.29       205
           4       0.58      0.42      0.49       186
           5       0.27      0.32      0.30       223
           6       0.27      0.29      0.28        41
           7       0.00      0.00      0.00        26
           8       0.31      0.14      0.20        90
           9       0.00      0.00      0.00        69
          10       0.19      0.05      0.08       107
          11       0.07      0.05      0.06        22
          12       0.35      0.46      0.40       176
          13       0.20      0.10      0.13       165
          14       0.59      0.69      0.64       342
          15       0.21      0.10      0.13       164

    accuracy                           0.35      2488
   macro avg       0.25   

/home/hanbiru/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hanbiru/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hanbiru/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
# !pip install pytorch-tabnet

In [57]:
from pytorch_tabnet.tab_model import TabNetClassifier

tabnet_clf = TabNetClassifier()
tabnet_clf.fit(X_train, y_train)

/home/hanbiru/anaconda3/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/home/hanbiru/anaconda3/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 2.96258 |  0:00:00s
epoch 1  | loss: 2.55152 |  0:00:00s
epoch 2  | loss: 2.42532 |  0:00:01s
epoch 3  | loss: 2.33411 |  0:00:01s
epoch 4  | loss: 2.26141 |  0:00:02s
epoch 5  | loss: 2.20928 |  0:00:02s
epoch 6  | loss: 2.17707 |  0:00:03s
epoch 7  | loss: 2.13826 |  0:00:03s
epoch 8  | loss: 2.12476 |  0:00:03s
epoch 9  | loss: 2.09715 |  0:00:04s
epoch 10 | loss: 2.07454 |  0:00:04s
epoch 11 | loss: 2.03398 |  0:00:05s
epoch 12 | loss: 2.01713 |  0:00:05s
epoch 13 | loss: 1.98747 |  0:00:06s
epoch 14 | loss: 1.99352 |  0:00:06s
epoch 15 | loss: 1.9777  |  0:00:06s
epoch 16 | loss: 1.96594 |  0:00:07s
epoch 17 | loss: 1.94915 |  0:00:07s
epoch 18 | loss: 1.95693 |  0:00:08s
epoch 19 | loss: 1.94656 |  0:00:08s
epoch 20 | loss: 1.93674 |  0:00:08s
epoch 21 | loss: 1.91979 |  0:00:09s
epoch 22 | loss: 1.91188 |  0:00:09s
epoch 23 | loss: 1.91787 |  0:00:09s
epoch 24 | loss: 1.91271 |  0:00:10s
epoch 25 | loss: 1.90021 |  0:00:10s
epoch 26 | loss: 1.89786 |  0:00:11s
e

In [58]:
tabnet_y_pred = tabnet_clf.predict(X_test)
tabnet_report = classification_report(y_test, tabnet_y_pred)
print(tabnet_report)

              precision    recall  f1-score   support

           0       0.34      0.40      0.37       305
           1       0.00      0.00      0.00        22
           2       0.42      0.70      0.53       345
           3       0.32      0.29      0.30       205
           4       0.59      0.51      0.55       186
           5       0.31      0.29      0.30       223
           6       0.26      0.37      0.30        41
           7       0.00      0.00      0.00        26
           8       0.30      0.22      0.26        90
           9       0.17      0.01      0.03        69
          10       0.23      0.09      0.13       107
          11       0.14      0.05      0.07        22
          12       0.39      0.45      0.42       176
          13       0.21      0.12      0.15       165
          14       0.72      0.71      0.71       342
          15       0.22      0.24      0.23       164

    accuracy                           0.41      2488
   macro avg       0.29   

/home/hanbiru/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hanbiru/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hanbiru/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
# !pip install xgboost

In [60]:
import xgboost as xgb
xgb_clf = xgb.XGBClassifier()
                            # tree_method="gpu_hist",
                            # early_stopping_rounds=20,
                            # eval_metric=['mlogloss'],
                            # enable_categorical=True,
                            # seed=42)


# classes_weights = class_weight.compute_sample_weight(
#     class_weight='balanced',
#     y=audio_data['genre'])

xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [61]:
xgb_y_pred = xgb_clf.predict(X_test)
xgb_report = classification_report(y_test, xgb_y_pred)
print(xgb_report)

              precision    recall  f1-score   support

           0       0.41      0.43      0.42       305
           1       0.25      0.05      0.08        22
           2       0.46      0.63      0.53       345
           3       0.27      0.30      0.28       205
           4       0.57      0.59      0.58       186
           5       0.30      0.35      0.33       223
           6       0.30      0.27      0.28        41
           7       0.00      0.00      0.00        26
           8       0.43      0.22      0.29        90
           9       0.19      0.07      0.11        69
          10       0.24      0.11      0.15       107
          11       0.17      0.05      0.07        22
          12       0.46      0.51      0.48       176
          13       0.17      0.15      0.16       165
          14       0.69      0.75      0.72       342
          15       0.23      0.20      0.21       164

    accuracy                           0.42      2488
   macro avg       0.32   

/home/hanbiru/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hanbiru/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hanbiru/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [74]:
best_XGB_params = {
                'max_depth': 4,
                'learning_rate': 0.03711299495104505,
                'n_estimators': 447,
                'min_child_weight': 10,
                'gamma': 0.002238238469617126,
                'subsample': 0.849826493051354,
                'colsample_bytree': 0.9977007889943236,
                'reg_alpha': 0.15538382519234734,
                'reg_lambda': 7.07955517130104e-07,
                'enable_categorical' : True,
                'tree_method' : "hist"
               }

xgb2_clf = xgb.XGBClassifier(**best_XGB_params)

xgb2_clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9977007889943236, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=0.002238238469617126, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.03711299495104505, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=10, missing=nan, monotone_constraints=None,
              n_estimators=447, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [75]:
xgb2_y_pred = xgb2_clf.predict(X_test)
xgb2_report = classification_report(y_test, xgb2_y_pred)
print(xgb2_report)

              precision    recall  f1-score   support

           0       0.44      0.48      0.46       305
           1       0.40      0.09      0.15        22
           2       0.45      0.67      0.54       345
           3       0.32      0.34      0.33       205
           4       0.55      0.61      0.58       186
           5       0.29      0.32      0.31       223
           6       0.30      0.34      0.32        41
           7       0.00      0.00      0.00        26
           8       0.50      0.24      0.33        90
           9       0.06      0.01      0.02        69
          10       0.20      0.10      0.14       107
          11       0.18      0.09      0.12        22
          12       0.46      0.52      0.49       176
          13       0.20      0.13      0.16       165
          14       0.72      0.77      0.75       342
          15       0.28      0.21      0.24       164

    accuracy                           0.44      2488
   macro avg       0.33   

/home/hanbiru/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hanbiru/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hanbiru/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [64]:
#!pip install imbalanced-learn==0.7.0

In [76]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)

X_res, y_res = sm.fit_resample(X_train, y_train)

xgb3_clf = xgb.XGBClassifier()#**best_XGB_params)

xgb3_clf.fit(X_res, y_res)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [77]:
xgb3_y_pred = xgb3_clf.predict(X_test)
xgb3_report = classification_report(y_test, xgb3_y_pred)
print(xgb3_report)

              precision    recall  f1-score   support

           0       0.46      0.38      0.42       305
           1       0.13      0.18      0.15        22
           2       0.48      0.53      0.50       345
           3       0.29      0.27      0.28       205
           4       0.53      0.55      0.54       186
           5       0.25      0.26      0.25       223
           6       0.22      0.34      0.27        41
           7       0.00      0.00      0.00        26
           8       0.33      0.29      0.31        90
           9       0.07      0.07      0.07        69
          10       0.24      0.21      0.22       107
          11       0.09      0.09      0.09        22
          12       0.45      0.49      0.47       176
          13       0.21      0.19      0.20       165
          14       0.74      0.71      0.72       342
          15       0.19      0.20      0.20       164

    accuracy                           0.39      2488
   macro avg       0.29   

In [78]:
tabnet2_clf = TabNetClassifier()
tabnet2_clf.fit(X_res, y_res)

/home/hanbiru/anaconda3/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/home/hanbiru/anaconda3/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 2.79921 |  0:00:01s
epoch 1  | loss: 2.48069 |  0:00:01s
epoch 2  | loss: 2.31711 |  0:00:02s
epoch 3  | loss: 2.23823 |  0:00:03s
epoch 4  | loss: 2.17958 |  0:00:04s
epoch 5  | loss: 2.13947 |  0:00:05s
epoch 6  | loss: 2.10044 |  0:00:06s
epoch 7  | loss: 2.07707 |  0:00:07s
epoch 8  | loss: 2.05519 |  0:00:08s
epoch 9  | loss: 2.04709 |  0:00:09s
epoch 10 | loss: 2.03045 |  0:00:09s
epoch 11 | loss: 2.01601 |  0:00:11s
epoch 12 | loss: 2.00572 |  0:00:12s
epoch 13 | loss: 1.99207 |  0:00:12s
epoch 14 | loss: 1.98351 |  0:00:13s
epoch 15 | loss: 1.96179 |  0:00:14s
epoch 16 | loss: 1.94959 |  0:00:15s
epoch 17 | loss: 1.93701 |  0:00:16s
epoch 18 | loss: 1.93451 |  0:00:17s
epoch 19 | loss: 1.93144 |  0:00:18s
epoch 20 | loss: 1.91482 |  0:00:19s
epoch 21 | loss: 1.89645 |  0:00:20s
epoch 22 | loss: 1.89656 |  0:00:21s
epoch 23 | loss: 1.8789  |  0:00:22s
epoch 24 | loss: 1.88147 |  0:00:23s
epoch 25 | loss: 1.87127 |  0:00:23s
epoch 26 | loss: 1.86774 |  0:00:24s
e

In [79]:
tabnet2_y_pred = tabnet2_clf.predict(X_test)
tabnet2_report = classification_report(y_test, tabnet2_y_pred)
print(tabnet2_report)

              precision    recall  f1-score   support

           0       0.41      0.28      0.33       305
           1       0.01      0.05      0.02        22
           2       0.42      0.54      0.48       345
           3       0.31      0.18      0.22       205
           4       0.60      0.41      0.49       186
           5       0.28      0.19      0.23       223
           6       0.14      0.41      0.21        41
           7       0.05      0.19      0.08        26
           8       0.29      0.32      0.31        90
           9       0.10      0.32      0.16        69
          10       0.25      0.24      0.24       107
          11       0.04      0.09      0.06        22
          12       0.42      0.28      0.34       176
          13       0.20      0.12      0.15       165
          14       0.81      0.58      0.68       342
          15       0.18      0.22      0.20       164

    accuracy                           0.33      2488
   macro avg       0.28   